# BackTesting 


In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-I.csv', parse_dates=['Date'], index_col=['Date'])
data = data.loc['2020-01-01':]
data

,Open,High,Low,Close,Adj Close,Volume,ema20,ema50,rsi,macd,macd_signal,Price_Up
Date,,,,,,,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997,7290.616103,7572.187445,45.182876,-61.114299,-85.231483,0
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465,7261.554590,7549.178926,39.271849,-79.166603,-84.018507,1
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032,7269.490751,7541.167371,50.861068,-63.736756,-79.962157,1
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275,7282.935130,7536.049307,52.642173,-45.674693,-73.104664,1
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095,7295.162012,7531.157859,52.660734,-30.950295,-64.673790,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-21,97756.195312,99507.101562,96426.523438,97224.726562,97224.726562,51765334294,99069.601344,92289.965450,48.952697,1965.864891,3012.484494,0
2024-12-22,97218.320312,97360.265625,94202.187500,95104.937500,95104.937500,43147981314,98692.014312,92400.356511,44.934433,1437.643624,2697.516320,0
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,94686.242188,65239002919,98310.512205,92489.999087,44.163328,974.011455,2352.815347,1


In [6]:
import torch
model = torch.load('model.pth')
model.eval()


C:\Users\Ivo\AppData\Local\Temp\ipykernel_5872\3331119187.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('model.pth')


AttributeError: Can't get attribute 'MiLSTM' on <module '__main__'>

In [38]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, plot_heatmaps
import talib 
import joblib
import torch
# Supongo que ya tienes un DataFrame `data` cargado con las columnas necesarias
features = ['Close', 'ema20', 'ema50', 'rsi', 'macd', 'macd_signal']
scaler = joblib.load('scaler.pkl')  # Cargamos el escalador previamente guardado
data_scaled = scaler.transform(data[features])  # Escalamos las características

class LSTMStrategy(Strategy):
    def init(self):
        # Cargar el modelo LSTM
        self.model = MiLSTM(input_size=6, hidden_size=128, num_layers=4, output_size=1)
        self.model.load_state_dict(torch.load('model.pth', map_location=torch.device('cpu')))
        self.model.eval()
        self.device = 'cpu'
        self.model.to(self.device)

    def next(self):
        # Preparar los datos para la predicción
        recent_data = data_scaled[len(self.data) - len(self.data.Close):]  # Últimos valores escalados
        input_tensor = torch.tensor(recent_data[-1], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(self.device)

        # Realizar predicción
        with torch.no_grad():
            prediction = torch.sigmoid(self.model(input_tensor)).item()

        price = self.data.Close[-1]  # Precio actual

        # Estrategia basada en predicción
        if prediction > 0.5 and not self.position.is_long:
            self.buy(sl=price * 0.95)
        elif prediction <= 0.5 and self.position.is_long:
            self.position.close()

        

bt = Backtest(data, SMAcross,cash=200000 )
bt.run()

C:\Users\Ivo\AppData\Local\Temp\ipykernel_17096\1947520989.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load('model.pth', map_locati

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
bt.plot()

RuntimeError: First issue `backtest.run()` to obtain results.

In [26]:
class RSI(Strategy):
    rsi_period = 22 # atributo de clase
    sl_percent = 0.95
    out_position = 30
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period) # accedo al atributo de la clase desde una instancia. 
        
    def next(self):
        price = self.data.Close[-1]

        if self.rsi[-1] < self.out_position:
            self.position.close()

        if self.position.is_long:
            return
        
        self.buy(sl= price*self.sl_percent)

bt = Backtest(data, RSI, cash=200000)
starts, heat = bt.optimize(rsi_period=range(5, 30, 1),
            sl_percent=[0.9,0.91,0.92,0.93,],
            maximize='Return [%]',
            return_heatmap=True)

starts


/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


Start                     2020-01-01 00:00:00
End                       2024-12-22 00:00:00
Duration                   1817 days 00:00:00
Exposure Time [%]                   99.394939
Equity Final [$]               4695090.119614
Equity Peak [$]                5122998.869614
Return [%]                         2247.54506
Buy & Hold Return [%]             1223.254205
Return (Ann.) [%]                   88.442243
Volatility (Ann.) [%]              122.062142
Sharpe Ratio                         0.724567
Sortino Ratio                        2.337413
Calmar Ratio                         1.305763
Max. Drawdown [%]                  -67.732218
Avg. Drawdown [%]                   -7.397474
Max. Drawdown Duration      824 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                                   84
Win Rate [%]                        39.285714
Best Trade [%]                     403.256672
Worst Trade [%]                      -7.87189
Avg. Trade [%]                    

In [37]:
starts['_strategy']
plot_heatmaps(heat)


GridPlot(id='p1896', ...)

In [8]:
bt.plot()

/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest valu

GridPlot(id='p1705', ...)